# Data Manipulation <a href=""><img src="../../../img/link-external.png" alt="external link" title="Go full page"/></a>

The previous notebook revealed the presence of incorrectly formatted and incomplete values in the dataset. This highlights the necessity of cleaning and transforming the data to ensure its seamless integration and enable a more targeted analysis.

This notebook covers the following topics:

- [Data Cleaning](#data-cleaning)
- [Data Transformation](#data-transformation)
  - [Sorting](#sorting)
  - [Query and Aggregation](#query-and-aggregation)
    - [Annual Dataframes](#annual-dataframes)
    - [Monthly Dataframes](#monthly-dataframes)
- [Questions and Answers](#questions-and-answers)
- [Bonus: Automate dataframes generation](#bonus:-automate-dataframes-generation)
- [Tests](#tests)

In [1]:
from typing import Dict, Literal, Optional
import calendar
import pandas as pd

In [2]:
volve_df = pd.read_excel(io="../../Data/raw-data/Volve production data.xlsx", sheet_name="Monthly Production Data")

In [3]:
volve_df

,Wellbore name,NPDCode,Year,Month,On Stream,Oil,Gas,Water,GI,WI
0,NaN,NaN,NaN,NaN,hrs,Sm3,Sm3,Sm3,Sm3,Sm3
1,15/9-F-1 C,7405.0,2014.0,4.0,227.5,11142.47,1597936.65,0,NaN,NaN
2,15/9-F-1 C,7405.0,2014.0,5.0,733.83334,24901.95,3496229.65,783.48,NaN,NaN
3,15/9-F-1 C,7405.0,2014.0,6.0,705.91666,19617.76,2886661.69,2068.48,NaN,NaN
4,15/9-F-1 C,7405.0,2014.0,7.0,742.41666,15085.68,2249365.75,6243.98,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
522,15/9-F-5,5769.0,2016.0,5.0,732,9724.4,1534677.16,3949.9,NaN,0
523,15/9-F-5,5769.0,2016.0,6.0,718.41667,9121.48,1468557.12,2376.93,NaN,NaN
524,15/9-F-5,5769.0,2016.0,7.0,668.64168,9985.29,1602674.39,2453.71,NaN,0
525,15/9-F-5,5769.0,2016.0,8.0,608.425,8928.9,1417278.51,2371.86,NaN,0


## Data Cleaning

To clean the dataset, we will proceed the following way:

- Format column names to `SCREAMING_SNAKE_CASE`
- Drop the first row
- Replace missing values with zero
- Make the necessary type conversions
- Change the `Month` column into month abbreviations (*1 -> Jan, 2 -> Feb, ...etc*)
- Create a function to automate the process

In [4]:
volve_df.columns = volve_df.columns.str.replace(" ", "_")
volve_df.columns = volve_df.columns.str.upper()

In [5]:
volve_df = volve_df.drop(index=[0]).reset_index(drop=True)

In [6]:
volve_df = volve_df.fillna(0)

In [7]:
volve_df[["NPDCODE", "YEAR", "MONTH"]] = volve_df[["NPDCODE", "YEAR", "MONTH"]].astype(int)
volve_df[["ON_STREAM", "OIL", "GAS", "WATER", "WI", "GI"]] = volve_df[["ON_STREAM", "OIL", "GAS", "WATER", "WI", "GI"]].astype(float)

In [8]:
abbreviations = dict(enumerate(calendar.month_abbr))
abbreviations

{0: '',
 1: 'Jan',
 2: 'Feb',
 3: 'Mar',
 4: 'Apr',
 5: 'May',
 6: 'Jun',
 7: 'Jul',
 8: 'Aug',
 9: 'Sep',
 10: 'Oct',
 11: 'Nov',
 12: 'Dec'}

In [9]:
abbreviations.pop(0)

''

In [10]:
volve_df["MONTH"] = volve_df["MONTH"].map(abbreviations)
volve_df["MONTH"] = pd.Categorical(volve_df["MONTH"], categories=list(abbreviations.values()), ordered=True)

In [11]:
def clean_data(data: pd.DataFrame) -> pd.DataFrame:
    df = data.copy()

    # Format column names to SCREAMING_SNAKE_CASE
    df.columns = df.columns.str.replace(" ", "_")
    df.columns = df.columns.str.upper()

    # To avoid confusion in case of a csv file
    df = df.replace(",", "", regex=True)

    df = df.drop(index=[0]).reset_index(drop=True)
    df = df.fillna(0)
    
    df[["NPDCODE", "YEAR", "MONTH"]] = df[["NPDCODE", "YEAR", "MONTH"]].astype(int)
    df[["ON_STREAM", "OIL", "GAS", "WATER", "WI", "GI"]] = df[["ON_STREAM", "OIL", "GAS", "WATER", "WI", "GI"]].astype(float)

    # Convert months to month abbreviations, 1 -> Jan, 2 -> Feb, ...etc
    abbr = dict(enumerate(calendar.month_abbr))
    abbr.pop(0)
    df["MONTH"] = df["MONTH"].map(abbr)
    df["MONTH"] = pd.Categorical(df["MONTH"], categories=list(abbr.values()), ordered=True)

    return df

In [12]:
volve_df.head(10)

,WELLBORE_NAME,NPDCODE,YEAR,MONTH,ON_STREAM,OIL,GAS,WATER,GI,WI
0,15/9-F-1 C,7405,2014,Apr,227.50000,11142.47,1597936.65,0.00,0.0,0.0
1,15/9-F-1 C,7405,2014,May,733.83334,24901.95,3496229.65,783.48,0.0,0.0
2,15/9-F-1 C,7405,2014,Jun,705.91666,19617.76,2886661.69,2068.48,0.0,0.0
3,15/9-F-1 C,7405,2014,Jul,742.41666,15085.68,2249365.75,6243.98,0.0,0.0
4,15/9-F-1 C,7405,2014,Aug,432.99166,6970.43,1048190.80,4529.75,0.0,0.0
5,15/9-F-1 C,7405,2014,Sep,630.30000,9168.43,1414099.99,8317.59,0.0,0.0
6,15/9-F-1 C,7405,2014,Oct,745.00000,9468.06,1462063.99,10364.87,0.0,0.0
7,15/9-F-1 C,7405,2014,Nov,579.77500,6710.33,1044188.30,7234.24,0.0,0.0
8,15/9-F-1 C,7405,2014,Dec,27.50000,120.29,25857.08,183.44,0.0,0.0
9,15/9-F-1 C,7405,2015,Jan,479.91667,10875.53,1604934.60,6850.80,0.0,0.0


[Back to Top](#data-manipulation)

## Data Transformation

The previous notebook revealed that the dataset has a temporal coverage of 9 years, going from **2007** to **2016**. However, the above cell shows the dataset starting from **2014**. This is because information were entered in the order `WELLBORE_NAME -> YEAR -> MONTH`.

To have the dataset to appear in a chronological order, we can always sort it by `YEAR` then by `MONTH`.

**Note:** This won't be necessary for the rest of the analysis, but I had to mention it.

### Sorting

In [13]:
volve_df = volve_df.sort_values(["YEAR", "MONTH"]).reset_index(drop=True)

In [14]:
volve_df

,WELLBORE_NAME,NPDCODE,YEAR,MONTH,ON_STREAM,OIL,GAS,WATER,GI,WI
0,15/9-F-4,5693,2007,Sep,0.00000,0.0,0.0,0.0,0.0,0.000000
1,15/9-F-5,5769,2007,Sep,0.00000,0.0,0.0,0.0,0.0,0.000000
2,15/9-F-4,5693,2007,Oct,0.00000,0.0,0.0,0.0,0.0,0.000000
3,15/9-F-5,5769,2007,Oct,0.00000,0.0,0.0,0.0,0.0,0.000000
4,15/9-F-4,5693,2007,Nov,0.00000,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...
521,15/9-F-4,5693,2016,Sep,340.10833,0.0,0.0,0.0,0.0,61200.875995
522,15/9-F-5,5769,2016,Sep,0.00000,0.0,0.0,0.0,0.0,0.000000
523,15/9-F-4,5693,2016,Oct,0.00000,0.0,0.0,0.0,0.0,0.000000
524,15/9-F-4,5693,2016,Nov,0.00000,0.0,0.0,0.0,0.0,0.000000


### Query and Aggregation

Now that the dataset is clean and sorted, creating smaller reusable chunks from it will help ensure a more targeted analysis.

To do so, we will proceed the following way:

- Create annual dataframes
- Add columns to account for the cummulated production
- Create monthly dataframes
- Create functions to automate the processes

#### Annual dataframes

In [15]:
annual_production_df = volve_df.groupby("YEAR", as_index=False)[["OIL", "GAS", "WATER"]].sum()

In [16]:
annual_production_df

,YEAR,OIL,GAS,WATER
0,2007,0.00,0.000000e+00,0.00
1,2008,1764375.15,2.570386e+08,23508.94
2,2009,2684392.04,3.759820e+08,232103.74
3,2010,1689902.55,2.439484e+08,1887594.78
4,2011,847965.40,1.303202e+08,2190640.28
5,2012,574206.33,9.020864e+07,2109601.63
6,2013,558012.53,8.758778e+07,2576365.79
7,2014,743107.49,1.117919e+08,2716070.92
8,2015,861749.06,1.284238e+08,2009825.86
9,2016,313370.06,5.006913e+07,1572866.41


In [17]:
annual_production_df["CUM_OIL"] = annual_production_df["OIL"].cumsum()
annual_production_df["CUM_GAS"] = annual_production_df["GAS"].cumsum()
annual_production_df["CUM_WATER"] = annual_production_df["WATER"].cumsum()

In [18]:
annual_production_df

,YEAR,OIL,GAS,WATER,CUM_OIL,CUM_GAS,CUM_WATER
0,2007,0.00,0.000000e+00,0.00,0.00,0.000000e+00,0.00
1,2008,1764375.15,2.570386e+08,23508.94,1764375.15,2.570386e+08,23508.94
2,2009,2684392.04,3.759820e+08,232103.74,4448767.19,6.330206e+08,255612.68
3,2010,1689902.55,2.439484e+08,1887594.78,6138669.74,8.769690e+08,2143207.46
4,2011,847965.40,1.303202e+08,2190640.28,6986635.14,1.007289e+09,4333847.74
5,2012,574206.33,9.020864e+07,2109601.63,7560841.47,1.097498e+09,6443449.37
6,2013,558012.53,8.758778e+07,2576365.79,8118854.00,1.185086e+09,9019815.16
7,2014,743107.49,1.117919e+08,2716070.92,8861961.49,1.296877e+09,11735886.08
8,2015,861749.06,1.284238e+08,2009825.86,9723710.55,1.425301e+09,13745711.94
9,2016,313370.06,5.006913e+07,1572866.41,10037080.61,1.475370e+09,15318578.35


In [19]:
annual_injection_df = volve_df.groupby("YEAR", as_index=False)[["GI", "WI"]].sum()

In [20]:
annual_injection_df

,YEAR,GI,WI
0,2007,0.0,0.000000e+00
1,2008,0.0,2.417770e+06
2,2009,0.0,4.392198e+06
3,2010,0.0,4.569394e+06
4,2011,0.0,3.442446e+06
5,2012,0.0,2.975214e+06
6,2013,0.0,3.453761e+06
7,2014,0.0,3.825493e+06
8,2015,0.0,3.201627e+06
9,2016,0.0,2.052229e+06


In [21]:
def get_annual_data(data: pd.DataFrame, category: Optional[Literal["production", "injection"]] = None) -> pd.DataFrame:
    
    df = data.groupby("YEAR", as_index=False)[["ON_STREAM", "OIL", "GAS", "WATER", "WI", "GI"]].sum()

    if not category:
        return df

    category = category.strip().lower()

    if category in ["production", "prod"]:
        df["CUM_OIL"] = df["OIL"].cumsum()
        df["CUM_GAS"] = df["GAS"].cumsum()
        df["CUM_WATER"] = df["WATER"].cumsum()
        df = df.drop(columns=["ON_STREAM", "GI", "WI"])

    elif category in ["injection", "inj"]:
        df = df[["YEAR", "GI", "WI"]]

    return df

#### Monthly dataframes

In [22]:
monthly_df = volve_df.groupby(["YEAR", "MONTH"], as_index=False)[["ON_STREAM", "OIL", "GAS", "WATER", "GI", "WI"]].sum()

In [23]:
monthly_df.head(20)

,YEAR,MONTH,ON_STREAM,OIL,GAS,WATER,GI,WI
0,2007,Jan,0.00000,0.00,0.00,0.00,0.0,0.000000
1,2007,Feb,0.00000,0.00,0.00,0.00,0.0,0.000000
2,2007,Mar,0.00000,0.00,0.00,0.00,0.0,0.000000
3,2007,Apr,0.00000,0.00,0.00,0.00,0.0,0.000000
4,2007,May,0.00000,0.00,0.00,0.00,0.0,0.000000
5,2007,Jun,0.00000,0.00,0.00,0.00,0.0,0.000000
6,2007,Jul,0.00000,0.00,0.00,0.00,0.0,0.000000
7,2007,Aug,0.00000,0.00,0.00,0.00,0.0,0.000000
8,2007,Sep,0.00000,0.00,0.00,0.00,0.0,0.000000
9,2007,Oct,0.00000,0.00,0.00,0.00,0.0,0.000000


In [24]:
monthly_oil_production_df = monthly_df.pivot_table("OIL", "MONTH", "YEAR")

In [25]:
monthly_oil_production_df

YEAR,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
MONTH,,,,,,,,,,
Jan,0.0,0.00,269249.82,231880.08,106075.00,45527.08,47593.37,61910.82,81246.37,51576.91
Feb,0.0,49091.06,235785.99,165118.98,73223.80,64159.92,41682.91,56956.10,66272.50,47367.82
Mar,0.0,83361.26,183603.15,168560.86,94105.01,61425.47,25122.33,62162.76,81739.35,42802.44
Apr,0.0,74532.45,237899.74,159289.42,89633.96,60598.14,35226.76,76198.75,90223.62,32310.04
May,0.0,125478.56,257214.05,164922.84,88326.33,57467.11,43974.93,83827.16,87064.14,39030.25
Jun,0.0,143786.85,219083.74,144927.19,53220.57,52552.93,34197.83,79538.14,83847.63,35659.34
Jul,0.0,166279.69,170977.07,133639.94,32673.73,54167.64,38387.49,74981.25,73995.25,31059.42
Aug,0.0,165443.66,224907.77,68531.09,86110.43,21688.08,62310.15,49375.58,56296.42,24954.66
Sep,0.0,192263.21,180448.69,106718.14,81052.30,2669.01,50869.33,62389.42,63097.13,8609.18


In [26]:
monthly_gas_production_df = monthly_df.pivot_table("GAS", "MONTH", "YEAR")

In [27]:
monthly_gas_production_df

YEAR,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
MONTH,,,,,,,,,,
Jan,0.0,0.00,38973679.35,32605905.94,16177205.30,6878732.17,7715896.16,9207271.11,11980379.17,8075648.69
Feb,0.0,7068009.29,33681428.98,23271838.23,11349336.66,9888483.76,6657580.37,8534705.15,10025732.79,7468629.01
Mar,0.0,12191170.91,25826010.21,23700324.16,14493974.18,9654374.59,3837458.41,9348667.90,12032559.44,6900266.92
Apr,0.0,11506440.74,32857763.99,22449616.35,13929703.75,9539380.01,5731548.81,11256658.87,13155179.90,5287855.05
May,0.0,19091871.52,35311402.26,23460677.21,13774667.97,9126346.20,7283687.29,11826032.32,12700588.28,6157954.41
Jun,0.0,21512333.93,29929356.76,20701111.07,8247937.92,8375057.64,5689962.72,11707630.91,12276974.83,5738861.41
Jul,0.0,24655302.85,23849827.69,19356911.12,4888317.28,8626031.17,6063603.54,11183245.92,10999937.32,4989889.87
Aug,0.0,23923540.58,31340971.77,9889756.53,13128124.42,3350388.60,9485747.35,7646877.09,8524505.93,3991207.77
Sep,0.0,27526459.16,25118054.93,16010763.07,12587790.94,402901.43,7816249.58,9788352.36,9518188.48,1458816.35


In [28]:
def get_monthly_data(
    data: pd.DataFrame, 
    parameter: Optional[Literal["ON_STREAM", "OIL", "GAS", "WATER", "GI", "WI"]] = None
) -> pd.DataFrame:
    
    df = data.groupby(["YEAR", "MONTH"], as_index=False)[["ON_STREAM", "OIL", "GAS", "WATER", "GI", "WI"]].sum()

    if not parameter:
        return df

    parameter = parameter.strip().upper().replace(" ", "_")
    df = df.pivot_table(values=parameter, index="MONTH", columns="YEAR")

    return df

[Back to Top](#data-manipulation)

## Questions and Answers

### Questions

- How many production wellbores are there in the dataset?
- How many injection wellbores are there in the dataset?

To answer these questions, let's create another data segment to store the computed yield of each wellbore.

In [29]:
wellbores_df = volve_df.groupby("WELLBORE_NAME", as_index=False)[["ON_STREAM", "OIL", "GAS", "WATER", "GI", "WI"]].sum()

In [30]:
wellbores_df

,WELLBORE_NAME,ON_STREAM,OIL,GAS,WATER,GI,WI
0,15/9-F-1 C,9983.53315,177709.33,2.644092e+07,207302.39,0.0,0.000000e+00
1,15/9-F-11,26006.21614,1147849.10,1.743105e+08,1090806.27,0.0,0.000000e+00
2,15/9-F-12,65204.06928,4579609.55,6.675423e+08,6833320.37,0.0,0.000000e+00
3,15/9-F-14,62773.67519,3942233.39,5.780095e+08,7121249.74,0.0,0.000000e+00
4,15/9-F-15 D,17824.83278,148518.56,2.250535e+07,52366.40,0.0,0.000000e+00
5,15/9-F-4,64267.16117,0.00,0.000000e+00,0.00,0.0,1.624073e+07
6,15/9-F-5,60829.85308,41160.68,6.561826e+06,13533.18,0.0,1.408941e+07


In [31]:
production_wells_df = wellbores_df.query("OIL > 0 or GAS > 0").reset_index(drop=True)

In [32]:
production_wells_df

,WELLBORE_NAME,ON_STREAM,OIL,GAS,WATER,GI,WI
0,15/9-F-1 C,9983.53315,177709.33,2.644092e+07,207302.39,0.0,0.000000e+00
1,15/9-F-11,26006.21614,1147849.10,1.743105e+08,1090806.27,0.0,0.000000e+00
2,15/9-F-12,65204.06928,4579609.55,6.675423e+08,6833320.37,0.0,0.000000e+00
3,15/9-F-14,62773.67519,3942233.39,5.780095e+08,7121249.74,0.0,0.000000e+00
4,15/9-F-15 D,17824.83278,148518.56,2.250535e+07,52366.40,0.0,0.000000e+00
5,15/9-F-5,60829.85308,41160.68,6.561826e+06,13533.18,0.0,1.408941e+07


In [33]:
print(f"There are {production_wells_df.shape[0]} production wellbores in the dataset.")

There are 6 production wellbores in the dataset.


In [34]:
injection_wells_df = wellbores_df.query("GI > 0 or WI > 0").reset_index(drop=True)

In [35]:
injection_wells_df

,WELLBORE_NAME,ON_STREAM,OIL,GAS,WATER,GI,WI
0,15/9-F-4,64267.16117,0.00,0.00,0.00,0.0,1.624073e+07
1,15/9-F-5,60829.85308,41160.68,6561825.74,13533.18,0.0,1.408941e+07


In [36]:
print(f"There are {injection_wells_df.shape[0]} injection wellbores in the dataset.")

There are 2 injection wellbores in the dataset.


### Answers

Out of the seven (07) wellbores:
- Six (06) were used for hydrocarbon production
- Two (02) were used for water injection during secondary recovery

However, `6+2=8` and the dataset only shows seven (07) wellbores. This implies that one of the wellbores was used for both hydrocarbon production and water injection.

Let's find out which wellbore it is.

In [37]:
production_wells_df.merge(injection_wells_df)

,WELLBORE_NAME,ON_STREAM,OIL,GAS,WATER,GI,WI
0,15/9-F-5,60829.85308,41160.68,6561825.74,13533.18,0.0,1.408941e+07


The wellbore `15/9-F-5` was used for both hydrocarbon production and water injection.

**Note:** There is no recognized term in the industry to label this kind of wellbore. So from now on, let's call it a **hybrid well**.

In [38]:
def wellbores_data(data: pd.DataFrame, category: Optional[Literal["production", "injection", "hybrid"]] = None) -> pd.DataFrame:
    
    df = data.groupby("WELLBORE_NAME", as_index=False)[["ON_STREAM", "OIL", "GAS", "WATER", "GI", "WI"]].sum()

    if not category:
        return df

    category = category.strip().lower()

    if category in ["production", "prod"]:
        df = df.query("OIL > 0 or GAS > 0")
    elif category in ["injection", "inj"]:
        df = df.query("GI > 0 or WI > 0")
    elif category in ["hybrid", "hb"]:
        df = df.query("(OIL > 0 or GAS > 0) and (GI > 0 or WI > 0)")

    df = df.reset_index(drop=True)

    return df

Knowing that the wellbore `15/9-F-5` was used for both hydrocarbon production and water injection, it would be legitimate to ask ourselves the following questions:

### Questions

- How long has the wellbore been used for hydrocarbon production?
- How long has it been used for water injection?

Moreover, in oil and gas production operations, not all wellbores are drilled early in the field's life. New wellbores may be drilled after the beginning of operations to increase production, and some may be shut down due to severe technical issues or to decrease production rates for economic reasons. Therefore, obtaining historical data for each wellbore can help provide a more accurate picture of events.

It is thus necessary to create individual subsets for each of the wellbores. As there are seven (07) wellbores in the dataset, creating subsets one by one would be tedious. Instead, let's create functions to automate the process.

In [39]:
def get_well_data(data: pd.DataFrame, well_name: str) -> pd.DataFrame:

    well_name = well_name.strip().upper()
    df = data.query("`WELLBORE_NAME` == @well_name").reset_index(drop=True)
    
    return df

In [40]:
def get_well_annual_data(
    data: pd.DataFrame, 
    well_name: str, 
    category: Optional[Literal["production", "injection"]] = None
) -> pd.DataFrame:
    
    df = get_well_data(data=data, well_name=well_name)
    df = df.groupby("YEAR", as_index=False)[["ON_STREAM", "OIL", "GAS", "WATER", "GI", "WI"]].sum()

    if not category:
        return df

    category = category.strip().lower()
    if category in ["production", "prod"]:
        df["CUM_OIL"] = df["OIL"].cumsum()
        df["CUM_GAS"] = df["GAS"].cumsum()
        df["CUM_WATER"] = df["WATER"].cumsum()
        df = df.drop(columns=["ON_STREAM", "GI", "WI"])

    elif category in ["injection", "inj"]:
        df = df[["YEAR", "GI", "WI"]]

    return df

In [41]:
# Note: If the argument 'parameter' is not provided, 
# the returned dataframe will be the same as the was obtained from the function 'get_well_data'

def get_well_monthly_data(
    data: pd.DataFrame, 
    well_name: str, 
    parameter: Optional[Literal["ON_STREAM", "OIL", "GAS", "WATER", "GI", "WI"]] = None
) -> pd.DataFrame:
    df = get_well_data(data=data, well_name=well_name)

    if not parameter:
        return df

    parameter = parameter.strip().upper().replace(" ", "_")
    df = df.pivot_table(values=parameter, index="MONTH", columns="YEAR", fill_value=0)

    return df

In [42]:
get_well_annual_data(data=volve_df, well_name="15/9-f-5")

,YEAR,ON_STREAM,OIL,GAS,WATER,GI,WI
0,2007,0.00000,0.00,0.00,0.00,0.0,0.000000e+00
1,2008,2678.37890,0.00,0.00,0.00,0.0,6.915290e+05
2,2009,7560.94328,0.00,0.00,0.00,0.0,2.242719e+06
3,2010,7852.04170,0.00,0.00,0.00,0.0,2.303923e+06
4,2011,6722.10837,0.00,0.00,0.00,0.0,1.721295e+06
5,2012,7057.86667,0.00,0.00,0.00,0.0,1.514296e+06
6,2013,7583.03501,0.00,0.00,0.00,0.0,1.788601e+06
7,2014,7783.07965,0.00,0.00,0.00,0.0,1.785109e+06
8,2015,8363.83279,0.00,0.00,0.00,0.0,1.569313e+06
9,2016,5228.56671,41160.68,6561825.74,13533.18,0.0,4.726207e+05


In [43]:
get_well_monthly_data(data=volve_df, well_name="15/9-f-5", parameter="wi")

YEAR,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
MONTH,,,,,,,,,,
Jan,0,0.000000,253655.425801,251554.767873,189041.119519,96145.249734,159939.686389,139161.964208,116851.662330,148394.556100
Feb,0,0.000000,226102.013160,199239.179513,139466.556430,144177.133349,147798.464818,139875.109217,122957.754830,164188.597139
Mar,0,0.000000,81674.350000,159955.292047,184592.187691,144901.311794,85998.065149,168354.400345,127803.687133,139810.591271
Apr,0,0.000000,104818.547780,143182.227376,188899.232610,152913.710861,121656.595674,170365.945108,146997.138336,20226.958790
May,0,0.000000,232334.514448,252211.737628,191438.083542,157759.447593,170935.465968,155443.530098,135253.884264,0.000000
Jun,0,0.000000,133204.205271,217995.582733,115810.712320,147174.916348,138629.205472,170548.081449,146025.800913,0.000000
Jul,0,0.000000,205447.018688,213816.939548,60234.670729,155141.689089,131248.921444,149633.537327,117659.770350,0.000000
Aug,0,14480.932077,198473.659979,120306.477756,161568.571770,65973.970819,185791.651601,122109.851845,113141.006224,0.000000
Sep,0,64814.286524,171773.204863,163185.260243,185636.237526,8067.451343,141322.974441,160662.468334,128260.871127,0.000000


In [44]:
get_well_monthly_data(data=volve_df, well_name="15/9-f-5", parameter="oil")

YEAR,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
MONTH,,,,,,,,,,
Jan,0,0,0,0,0,0,0,0,0,0.00
Feb,0,0,0,0,0,0,0,0,0,0.00
Mar,0,0,0,0,0,0,0,0,0,0.00
Apr,0,0,0,0,0,0,0,0,0,3400.61
May,0,0,0,0,0,0,0,0,0,9724.40
Jun,0,0,0,0,0,0,0,0,0,9121.48
Jul,0,0,0,0,0,0,0,0,0,9985.29
Aug,0,0,0,0,0,0,0,0,0,8928.90
Sep,0,0,0,0,0,0,0,0,0,0.00


### Answers

The wellbore `15/9-F-5` was primarily used as an injection wellbore, from `August 2008` to `April 2016`, before being converted into a production wellbore until `August 2016`.

In [45]:
def determine_well_type(data: pd.DataFrame, well_name: str) -> str:
     well_df = get_well_data(data, well_name)

     if sum(well_df["OIL"] + well_df["GAS"]) > 0:
          if sum(well_df["GI"] + well_df["WI"]) > 0:
               return "HYBRID"
          else:
               return "PRODUCTION"
     else:
          return "INJECTION"

In [46]:
def wellbores_details(data: pd.DataFrame) -> pd.DataFrame:
    details_dict = {
        "WELLBORE_NAME": [],
        "WELLBORE_TYPE": [],
        "FIRST_RECORD": [],
        "LAST_RECORD": [],
    }

    for wellbore in data["WELLBORE_NAME"].unique():
        well_df = get_well_data(data, wellbore)
        details_dict["WELLBORE_NAME"].append(wellbore)
        details_dict["WELLBORE_TYPE"].append(determine_well_type(data, wellbore))
        details_dict["FIRST_RECORD"].append(well_df["YEAR"].min())
        details_dict["LAST_RECORD"].append(well_df["YEAR"].max())
    
    return pd.DataFrame(details_dict).sort_values("FIRST_RECORD").reset_index(drop=True)


In [47]:
wellbores_details(volve_df)

,WELLBORE_NAME,WELLBORE_TYPE,FIRST_RECORD,LAST_RECORD
0,15/9-F-4,INJECTION,2007,2016
1,15/9-F-5,HYBRID,2007,2016
2,15/9-F-12,PRODUCTION,2008,2016
3,15/9-F-14,PRODUCTION,2008,2016
4,15/9-F-11,PRODUCTION,2013,2016
5,15/9-F-15 D,PRODUCTION,2014,2016
6,15/9-F-1 C,PRODUCTION,2014,2016


### Observations

The above dataframe reveals that out of seven (07) wellbores:
- Four (04) started operating at the beginning of the field's life (*2007 - 2008*)
- Three (03) were drilled years later, probably to increase the production
- All wellbores stopped operating in **2016**

**Note:** The graphical analysis of the annual production in the next notebook may reveal an increase in production around **2014**.

In [48]:
wellbores_details(volve_df).merge(wellbores_df, on="WELLBORE_NAME")

,WELLBORE_NAME,WELLBORE_TYPE,FIRST_RECORD,LAST_RECORD,ON_STREAM,OIL,GAS,WATER,GI,WI
0,15/9-F-4,INJECTION,2007,2016,64267.16117,0.00,0.000000e+00,0.00,0.0,1.624073e+07
1,15/9-F-5,HYBRID,2007,2016,60829.85308,41160.68,6.561826e+06,13533.18,0.0,1.408941e+07
2,15/9-F-12,PRODUCTION,2008,2016,65204.06928,4579609.55,6.675423e+08,6833320.37,0.0,0.000000e+00
3,15/9-F-14,PRODUCTION,2008,2016,62773.67519,3942233.39,5.780095e+08,7121249.74,0.0,0.000000e+00
4,15/9-F-11,PRODUCTION,2013,2016,26006.21614,1147849.10,1.743105e+08,1090806.27,0.0,0.000000e+00
5,15/9-F-15 D,PRODUCTION,2014,2016,17824.83278,148518.56,2.250535e+07,52366.40,0.0,0.000000e+00
6,15/9-F-1 C,PRODUCTION,2014,2016,9983.53315,177709.33,2.644092e+07,207302.39,0.0,0.000000e+00


### Question

- What could have motivated the company to increase the production?

### Answer

>#### Oil market chronology
>
>- According to the [U.S. Energy Information Administration](https://www.eia.gov/todayinenergy/detail.php?id=7630#:~:text=Crude%20oil%20prices%20rose%20during,start%20of%20the%20third%20quarter.), &ldquo;<cite>Crude oil prices rose during the first quarter of 2012 as concerns about possible international supply disruptions pushed up petroleum prices. Prices then fell during the second quarter before turning sharply upward at the start of the third quarter.</cite>&rdquo;
>
>- In **2013**, Brent crude rose above &dollar;110 and the [International Energy Agency](https://en.wikipedia.org/wiki/International_Energy_Agency) said demand for oil would still rise. <cite>(Source: [Wikipedia](https://en.wikipedia.org/wiki/2011%E2%80%932013_world_oil_market_chronology#:~:text=Oil%20supplies%20remained%20high%2C%20and,years%2C%20with%20gasoline%20prices%20following.))</cite>
>
><br>
>
>#### Annual Reports
>
>The [2013 Annual Report](https://cdn.equinor.com/files/h61q9gi9/global/f7148331f9bfa5ddb7b7b9851bc94e9bff66dfb0.pdf?statoil-annual-report-20-F-2013.pdf) from **Equinor/Statoil** states that:
>- &ldquo;<cite>Volve has executed a drilling programme in 2013. The drilling of well F11-A, and a successful exploration pilot at Volve North West, have increased the proved reserves. As a result the field life has been extended by two years, and production is now expected to run to the third quarter of 2016.</cite>&rdquo; (page 20)
>
><br>
>
>Moreover, the [2014 Annual Report](https://cdn.equinor.com/files/h61q9gi9/global/b2c0248e609fa8783137514d8fd0ca09f9a1ec5e.pdf?Statoil-20-F-2014.pdf) states that:
>- **Page 9**: &ldquo;<cite>After more than three years of relatively stable prices, 2014 saw the price of Brent crude climb to USD 115 per barrel in June before dropping to USD 55 per barrel at the end of December.</cite>&rdquo; <br><br>
>- **Page 23**: &ldquo;<cite>Volve (Statoil interest 59.60%) has successfully increased the proven reserve via a drilling program in 2014.</cite>&rdquo; <br><br>

#### Conclusion

With the regular rise of oil prices from **2012** and larger hydrocarbon reserves, the company may have decided to increase production to maximize its profits. The Annual Reports also confirm our understanding of the chronology of events during field development.

[Back to Top](#data-manipulation)

## Bonus: Automate dataframes generation

When graphically analyzing the dataset in the next notebook, it will be essential to generate numerous subsets to account for various data classifications. This could result in writing multiple lines to generate dataframes of the same scope (*annual for example*). It would thus be time-saving to have a function capable of generating all these, allowing us to access multiple dataframes in a single line through [unpacking](https://www.w3schools.com/python/python_tuples_unpack.asp).

In [49]:
def annual_data(
    data: pd.DataFrame, 
    well_name: Optional[str] = None, 
    category: Optional[Literal["production", "injection"]] = None
) -> pd.DataFrame:
    if well_name:
        return get_well_annual_data(data, well_name, category)
    else:
        return get_annual_data(data, category)

In [50]:
def monthly_data(
    data: pd.DataFrame,
    well_name: Optional[str] = None,
    parameter: Optional[Literal["ON_STREAM", "OIL", "GAS", "WATER", "GI", "WI"]] = None
) -> pd.DataFrame:
    if well_name:
        return get_well_monthly_data(data, well_name, parameter)
    else:
        return get_monthly_data(data, parameter)

⚠️`WARNING!:` Python dictionaries are case sensitive. Be careful when using the [**`get()`**](https://www.w3schools.com/python/ref_dictionary_get.asp) method to withdraw subsets.

In [51]:
def generate_annual_dataframes(data: pd.DataFrame, well_name: Optional[str] = None) -> Dict[str, pd.DataFrame]:
    dataframes_collection = {}

    dataframes_collection["PRODUCTION"] = annual_data(data, well_name, category="production")
    dataframes_collection["INJECTION"] = annual_data(data, well_name, category="injection")
    
    return dataframes_collection

In [52]:
def generate_monthly_dataframes(data: pd.DataFrame, well_name: Optional[str] = None) -> Dict[str, pd.DataFrame]:
    dataframes_collection = {}
    parameters = ["ON_STREAM", "OIL", "GAS", "WATER", "GI", "WI"]

    for parameter in parameters:
        dataframes_collection[parameter] = monthly_data(data, well_name, parameter)

    return dataframes_collection

In [53]:
def generate_wellbores_dataframes(data: pd.DataFrame) -> Dict[str, pd.DataFrame]:
    dataframes_collection = {}
    categories = ["PRODUCTION", "INJECTION", "HYBRID"]

    for category in categories:
        dataframes_collection[category] = wellbores_data(data, category)

    return dataframes_collection

[Back to Top](#data-manipulation)

## Tests

Feel free to test the functions in the cells below.

In [54]:
annual_prod_df, annual_inj_df = generate_annual_dataframes(data=volve_df).values()

In [55]:
annual_prod_df

,YEAR,OIL,GAS,WATER,CUM_OIL,CUM_GAS,CUM_WATER
0,2007,0.00,0.000000e+00,0.00,0.00,0.000000e+00,0.00
1,2008,1764375.15,2.570386e+08,23508.94,1764375.15,2.570386e+08,23508.94
2,2009,2684392.04,3.759820e+08,232103.74,4448767.19,6.330206e+08,255612.68
3,2010,1689902.55,2.439484e+08,1887594.78,6138669.74,8.769690e+08,2143207.46
4,2011,847965.40,1.303202e+08,2190640.28,6986635.14,1.007289e+09,4333847.74
5,2012,574206.33,9.020864e+07,2109601.63,7560841.47,1.097498e+09,6443449.37
6,2013,558012.53,8.758778e+07,2576365.79,8118854.00,1.185086e+09,9019815.16
7,2014,743107.49,1.117919e+08,2716070.92,8861961.49,1.296877e+09,11735886.08
8,2015,861749.06,1.284238e+08,2009825.86,9723710.55,1.425301e+09,13745711.94
9,2016,313370.06,5.006913e+07,1572866.41,10037080.61,1.475370e+09,15318578.35


In [56]:
annual_inj_df

,YEAR,GI,WI
0,2007,0.0,0.000000e+00
1,2008,0.0,2.417770e+06
2,2009,0.0,4.392198e+06
3,2010,0.0,4.569394e+06
4,2011,0.0,3.442446e+06
5,2012,0.0,2.975214e+06
6,2013,0.0,3.453761e+06
7,2014,0.0,3.825493e+06
8,2015,0.0,3.201627e+06
9,2016,0.0,2.052229e+06


In [57]:
f5_monthly_oil, f5_monthly_wi = (generate_monthly_dataframes(volve_df, well_name="15/9-F-5").get(key) for key in ("OIL", "WI"))

In [58]:
f5_monthly_oil

YEAR,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
MONTH,,,,,,,,,,
Jan,0,0,0,0,0,0,0,0,0,0.00
Feb,0,0,0,0,0,0,0,0,0,0.00
Mar,0,0,0,0,0,0,0,0,0,0.00
Apr,0,0,0,0,0,0,0,0,0,3400.61
May,0,0,0,0,0,0,0,0,0,9724.40
Jun,0,0,0,0,0,0,0,0,0,9121.48
Jul,0,0,0,0,0,0,0,0,0,9985.29
Aug,0,0,0,0,0,0,0,0,0,8928.90
Sep,0,0,0,0,0,0,0,0,0,0.00


In [59]:
f5_monthly_wi

YEAR,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
MONTH,,,,,,,,,,
Jan,0,0.000000,253655.425801,251554.767873,189041.119519,96145.249734,159939.686389,139161.964208,116851.662330,148394.556100
Feb,0,0.000000,226102.013160,199239.179513,139466.556430,144177.133349,147798.464818,139875.109217,122957.754830,164188.597139
Mar,0,0.000000,81674.350000,159955.292047,184592.187691,144901.311794,85998.065149,168354.400345,127803.687133,139810.591271
Apr,0,0.000000,104818.547780,143182.227376,188899.232610,152913.710861,121656.595674,170365.945108,146997.138336,20226.958790
May,0,0.000000,232334.514448,252211.737628,191438.083542,157759.447593,170935.465968,155443.530098,135253.884264,0.000000
Jun,0,0.000000,133204.205271,217995.582733,115810.712320,147174.916348,138629.205472,170548.081449,146025.800913,0.000000
Jul,0,0.000000,205447.018688,213816.939548,60234.670729,155141.689089,131248.921444,149633.537327,117659.770350,0.000000
Aug,0,14480.932077,198473.659979,120306.477756,161568.571770,65973.970819,185791.651601,122109.851845,113141.006224,0.000000
Sep,0,64814.286524,171773.204863,163185.260243,185636.237526,8067.451343,141322.974441,160662.468334,128260.871127,0.000000
